In [8]:
import torch
from pathlib import Path
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

# --------------------
# настройки
# --------------------
MODEL_PT = Path("C:/Users/Вячеслав/Documents/superresolution/models/best_x2_ms_resunet.pt")
IMG_PATH = Path('C:/Users/Вячеслав/Documents/superresolution/DeepRockSR-2D/carbonate2D/carbonate2D_test_LR_default_X2/3607x2.png')
OUT_PATH = Path('C:/Users/Вячеслав/Documents/superresolution/sr.png')

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# если у тебя grayscale SR: C=1; если RGB: C=3
FORCE_GRAYSCALE = True

# нормализация: чаще всего для SR используют [0,1]
# если при обучении делал mean/std — нужно повторить ТОЧНО так же.
USE_MEAN_STD = False
MEAN = (0.5,)   # пример для 1 канала
STD  = (0.5,)

# --------------------
# util: PIL -> tensor  (1,C,H,W)
# --------------------
def pil_to_tensor(img: Image.Image) -> torch.Tensor:
    if FORCE_GRAYSCALE:
        img = img.convert("L")  # 1 канал
        arr = np.array(img, dtype=np.float32) / 255.0          # H,W
        t = torch.from_numpy(arr)[None, None, :, :]            # 1,1,H,W
        if USE_MEAN_STD:
            t = (t - MEAN[0]) / STD[0]
        return t
    else:
        img = img.convert("RGB")
        arr = np.array(img, dtype=np.float32) / 255.0          # H,W,3
        t = torch.from_numpy(arr).permute(2,0,1)[None, :, :, :]# 1,3,H,W
        if USE_MEAN_STD:
            mean = torch.tensor(MEAN).view(1,3,1,1)
            std  = torch.tensor(STD).view(1,3,1,1)
            t = (t - mean) / std
        return t

# --------------------
# util: tensor -> PIL
# --------------------
def tensor_to_pil(t: torch.Tensor) -> Image.Image:
    # ожидаем 1,C,H,W или C,H,W
    if t.ndim == 4:
        t = t[0]
    t = t.detach().float().cpu()

    if USE_MEAN_STD:
        if t.shape[0] == 1:
            t = t * STD[0] + MEAN[0]
        else:
            mean = torch.tensor(MEAN).view(3,1,1)
            std  = torch.tensor(STD).view(3,1,1)
            t = t * std + mean

    t = t.clamp(0, 1)

    if t.shape[0] == 1:
        arr = (t[0].numpy() * 255.0).round().astype(np.uint8)  # H,W
        return Image.fromarray(arr, mode="L")
    else:
        arr = (t.permute(1,2,0).numpy() * 255.0).round().astype(np.uint8)  # H,W,3
        return Image.fromarray(arr, mode="RGB")

In [9]:
%cd C:\Users\Вячеслав\Documents\superresolution\modules

C:\Users\Вячеслав\Documents\superresolution\modules


In [10]:
import os, sys
print("cwd:", os.getcwd())
print("python:", sys.executable)
print("first sys.path entries:", sys.path[:5])

cwd: C:\Users\Вячеслав\Documents\superresolution\modules
python: C:\anaconda\envs\superresolution\python.exe
first sys.path entries: ['C:\\anaconda\\envs\\superresolution\\python313.zip', 'C:\\anaconda\\envs\\superresolution\\DLLs', 'C:\\anaconda\\envs\\superresolution\\Lib', 'C:\\anaconda\\envs\\superresolution', '']


In [12]:
import torch
from collections import OrderedDict

# 1) Импорт архитектуры (ВАЖНО: поправь импорт под свой файл)
# Ты писал, что архитектура у тебя в стиле:
# def MS_ResUNet():
#     model = RefineNet(Bottleneck, [3, 4, 3, 3])
#     return model
#
# Значит нужно импортировать RefineNet и Bottleneck оттуда, где они реально лежат.
# Примеры (выбери один правильный для твоего проекта):
# from refinenet import RefineNet, Bottleneck
# from model import RefineNet, Bottleneck
# from networks.refinenet import RefineNet, Bottleneck

from ms_resunet import RefineNet, Bottleneck  # <-- ПОДСТАВЬ СВОЙ ПРАВИЛЬНЫЙ ПУТЬ

def MS_ResUNet():
    return RefineNet(Bottleneck, [3, 4, 3, 3])

def _extract_state_dict(ckpt):
    """Достаёт state_dict из разных форматов чекпоинта."""
    if isinstance(ckpt, torch.nn.Module):
        # torch.save(model)
        return ckpt.state_dict()

    if not isinstance(ckpt, dict):
        # редкий кейс: сохранён просто OrderedDict без dict-обёртки
        if isinstance(ckpt, (OrderedDict, dict)):
            return ckpt
        raise TypeError(f"Неожиданный тип чекпоинта: {type(ckpt)}")

    # самые частые варианты:
    for key in ["model", "state_dict", "model_state_dict", "net", "generator", "g"]:
        if key in ckpt and isinstance(ckpt[key], (dict, OrderedDict)):
            return ckpt[key]

    # иногда сам dict и есть state_dict (ключи вида 'layer.weight')
    if any(isinstance(k, str) and k.endswith((".weight", ".bias")) for k in ckpt.keys()):
        return ckpt

    # покажем ключи, чтобы быстро понять формат
    raise KeyError(f"Не нашёл state_dict в чекпоинте. Ключи верхнего уровня: {list(ckpt.keys())[:50]}")

def _strip_prefix(state, prefix="module."):
    """Убирает 'module.' после DDP/DataParallel."""
    if not any(k.startswith(prefix) for k in state.keys()):
        return state
    out = OrderedDict()
    for k, v in state.items():
        out[k[len(prefix):] if k.startswith(prefix) else k] = v
    return out

# 2) Создаём модель
model = MS_ResUNet()

# 3) Грузим чекпоинт
ckpt = torch.load(MODEL_PT, map_location="cpu")
state = _extract_state_dict(ckpt)
state = _strip_prefix(state, "module.")

# 4) Загружаем веса
missing, unexpected = model.load_state_dict(state, strict=True)

print("Loaded:", MODEL_PT)
print("Missing:", len(missing))
print("Unexpected:", len(unexpected))
print("Missing examples:", missing[:20])
print("Unexpected examples:", unexpected[:20])

# 5) На устройство и в eval
model = model.to(DEVICE).eval()

Loaded: C:\Users\Вячеслав\Documents\superresolution\models\best_x2_ms_resunet.pt
Missing: 0
Unexpected: 0
Missing examples: []
Unexpected examples: []


In [13]:
img = Image.open(IMG_PATH)
x = pil_to_tensor(img).to(DEVICE)

with torch.no_grad():
    y = model(x)

# иногда модель возвращает tuple/list (например, (sr, aux))
if isinstance(y, (tuple, list)):
    y = y[0]

out_img = tensor_to_pil(y)
out_img.save(OUT_PATH)

print("Saved:", OUT_PATH)

Saved: C:\Users\Вячеслав\Documents\superresolution\sr.png


C:\Users\Вячеслав\AppData\Local\Temp\ipykernel_26232\2155808775.py:67: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(arr, mode="L")


In [14]:
print("x device:", x.device)
print("model device:", next(model.parameters()).device)

x device: cuda:0
model device: cuda:0


In [15]:
import torch
from PIL import Image
from torchvision.transforms import v2 as T
from torchvision.transforms.v2 import functional as TF
from torchvision.transforms.v2.functional import InterpolationMode

DEVICE   = "cuda" if torch.cuda.is_available() else "cpu"

LR_PATH  = Path('C:/Users/Вячеслав/Documents/superresolution/DeepRockSR-2D/carbonate2D/carbonate2D_test_LR_default_X2/3607x2.png')
HR_PATH  = Path('C:/Users/Вячеслав/Documents/superresolution/DeepRockSR-2D/carbonate2D/carbonate2D_test_HR/3607.png')
OUT_PATH = Path('C:/Users/Вячеслав/Documents/superresolution/sr.png')

# --- те же преобразования, что в val ---
to_img = T.ToImage()
to_f32 = T.ToDtype(torch.float32, scale=True)

lr = Image.open(LR_PATH).convert("L")
hr = Image.open(HR_PATH).convert("L")

if lr.size != hr.size:  # PIL size=(W,H)
    lr = TF.resize(lr, size=[hr.size[1], hr.size[0]], interpolation=InterpolationMode.BICUBIC, antialias=True)

x = to_f32(to_img(lr)).unsqueeze(0).to(DEVICE)   # (1,1,H,W) float32 в [0,1]

model = model.to(DEVICE).eval()
with torch.no_grad():
    y = model(x)
if isinstance(y, (tuple, list)):
    y = y[0]

# сохранить (ожидается, что y тоже в [0,1])
img_out = (y[0,0].detach().clamp(0,1).cpu().numpy() * 255).round().astype("uint8")
Image.fromarray(img_out, mode="L").save(OUT_PATH)
print("Saved:", OUT_PATH)

Saved: C:\Users\Вячеслав\Documents\superresolution\sr.png


C:\Users\Вячеслав\AppData\Local\Temp\ipykernel_26232\2497787645.py:33: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  Image.fromarray(img_out, mode="L").save(OUT_PATH)


In [ ]:
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont

# ===================== НАСТРОЙКИ =====================
FOLDER = Path("C:/Users/Вячеслав/Documents/superresolution/report")
BASE_ID = None          # например "3607" или None (тогда сделает отчёты по всем найденным id)
OUT_DIR = FOLDER        # куда сохранять отчёты
GRID = 4                # 4x4 = 16 патчей
RECT_WIDTH = 4
PAD = 12                # отступы между плитками
TITLE_H = 100          # высота области под подписи в каждой ячейке
# =====================================================

from PIL import ImageFont
from pathlib import Path

def pick_font(size=90):
    font_path = Path(r"C:\Windows\Fonts\arial.ttf")
    return ImageFont.truetype(str(font_path), size=size)

FONT = pick_font(70)

def find_triplets(folder: Path):
    """Ищем базовые id, у которых есть *_hr, *_lr, *_sr (с любым расширением)."""
    hr = {p.stem[:-3]: p for p in folder.glob("*_hr.*")}  # stem: 3607_hr -> base: 3607
    lr = {p.stem[:-3]: p for p in folder.glob("*_lr.*")}
    sr = {p.stem[:-3]: p for p in folder.glob("*_sr.*")}
    bases = sorted(set(hr) & set(lr) & set(sr))
    return [(b, hr[b], lr[b], sr[b]) for b in bases]

def tile_with_title(img: Image.Image, title: str, cell_w: int, cell_h: int):
    """Картинка + подпись снизу (внутри одной ячейки)."""
    canvas = Image.new("RGB", (cell_w, cell_h + TITLE_H), (20, 20, 20))
    # если img не RGB — конвертируем для вставки
    if img.mode != "RGB":
        img = img.convert("RGB")
    canvas.paste(img, (0, 0))

    draw = ImageDraw.Draw(canvas)
    draw.text((8, cell_h + 5), title, fill=(240, 240, 240), font=FONT)
    return canvas

def make_report_for_id(base: str, hr_path: Path, lr_path: Path, sr_path: Path, out_path: Path):
    hr = Image.open(hr_path)
    lr = Image.open(lr_path)
    sr = Image.open(sr_path)

    # Приводим к grayscale/или оставляем как есть — тут чаще всего grayscale.
    # Если у тебя цвет, можешь убрать convert("L").
    hr = hr.convert("L")
    lr = lr.convert("L")
    sr = sr.convert("L")

    W, H = hr.size
    assert W % GRID == 0 and H % GRID == 0, f"HR размер {W}x{H} должен делиться на GRID={GRID}"
    pw, ph = W // GRID, H // GRID

    # Чтобы патчи совпадали по координатам — LR/SR ресайзим под HR (если отличаются)
    if lr.size != hr.size:
        lr = lr.resize((W, H), resample=Image.BICUBIC)
    if sr.size != hr.size:
        sr = sr.resize((W, H), resample=Image.BICUBIC)

    # Размер одной "картинки" в ячейке: показываем патчи в масштабе full-size HR
    CELL_W, CELL_H = W, H

    rows = GRID * GRID
    cols = 4
    out_w = PAD + cols * (CELL_W + PAD)
    out_h = PAD + rows * (CELL_H + TITLE_H + PAD)

    report = Image.new("RGB", (out_w, out_h), (15, 15, 15))
    draw_global = ImageDraw.Draw(report)

    # Сдвиг вниз чуть-чуть, чтобы заголовок не лип к первому ряду
    y0_base = PAD + 24

    idx = 0
    for r in range(GRID):
        for c in range(GRID):
            x0, y0 = c * pw, r * ph
            x1, y1 = x0 + pw, y0 + ph

            hr_patch = hr.crop((x0, y0, x1, y1)).resize((CELL_W, CELL_H), resample=Image.NEAREST)
            lr_patch = lr.crop((x0, y0, x1, y1)).resize((CELL_W, CELL_H), resample=Image.NEAREST)
            sr_patch = sr.crop((x0, y0, x1, y1)).resize((CELL_W, CELL_H), resample=Image.NEAREST)

            # HR full с красной рамкой патча
            hr_box = hr.convert("RGB")
            d = ImageDraw.Draw(hr_box)
            d.rectangle([x0, y0, x1 - 1, y1 - 1], outline=(255, 0, 0), width=RECT_WIDTH)

            # Подписи
            t_hr = tile_with_title(hr_patch, f"HR patch [{r},{c}]", CELL_W, CELL_H)
            t_lr = tile_with_title(lr_patch, f"LR patch [{r},{c}]", CELL_W, CELL_H)
            t_sr = tile_with_title(sr_patch, f"SR patch [{r},{c}]", CELL_W, CELL_H)
            t_full = tile_with_title(hr_box,   f"HR full", CELL_W, CELL_H)

            # Позиции в отчёте
            row_y = y0_base + idx * (CELL_H + TITLE_H + PAD)

            def paste_col(col_i, tile):
                x = PAD + col_i * (CELL_W + PAD)
                report.paste(tile, (x, row_y))

            paste_col(0, t_hr)
            paste_col(1, t_lr)
            paste_col(2, t_sr)
            paste_col(3, t_full)

            idx += 1

    out_path.parent.mkdir(parents=True, exist_ok=True)
    if out_path.suffix == "":
        out_path = out_path.with_suffix(".png")
    report.save(out_path)
    return out_path

# ---------- ЗАПУСК ----------
triplets = find_triplets(FOLDER)
if not triplets:
    raise FileNotFoundError("Не нашёл в папке тройки *_hr.*, *_lr.*, *_sr.*")

targets = triplets
if BASE_ID is not None:
    targets = [t for t in triplets if t[0] == BASE_ID]
    if not targets:
        raise FileNotFoundError(f"Не нашёл тройку для BASE_ID={BASE_ID}")

saved_paths = []
for base, hr_p, lr_p, sr_p in targets:
    out = OUT_DIR / f"{base}_patch_report.png"
    saved = make_report_for_id(base, hr_p, lr_p, sr_p, out)
    saved_paths.append(saved)
    print("Saved:", saved)

saved_paths